In [ ]:
import os
from ipywidgets import Video

from utils.es_utils import index_training_folder

# Visualizing a TrainingRun

First simply create a TrainingRun object from the folder containing the training data you want to visualize.

In [ ]:
training_run = index_training_folder("path-to-folder")

Then specify the generation or use `-1` to get the latest generation with a model file.

In [ ]:
video_file = training_run.visualize(env_seed=None, generation=-1, force=False)

After the video is recorded (this can take some time in PyBullet) visualize it by creating a Video object and display it.
If it cannot be displayed run the recording cell again with `force=True`. Sometimes the OpenAI Gym does not render it properly.

In [ ]:
video = Video.from_file(video_file)
display(video)